In [21]:
import requests # for web scraping
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np

# sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from sentiment_dictionary import NEW_WORDS

# install yahoo finance
import yfinance as yf

from config import save_path

# import functions
from functions import scrape_finfiz

ImportError: cannot import name 'scrape_finfiz' from 'functions' (/Users/kacikus/Dropbox/stock-predictor/functions.py)

In [14]:
news = scrape_finviz(['AAPL'])

1 companies to gather data for
1


In [15]:
news

,company,date,time,headline
0,AAPL,Apr-21-21,02:21PM,UPDATE 1-Facebook tweaks ad tools ahead of App...
1,AAPL,Apr-21-21,02:17PM,FTC Nominee Khan Calls for More Scrutiny of Te...
2,AAPL,Apr-21-21,02:03PM,"UPDATE 1-Epic CEO, Apple App Store chief to at..."
3,AAPL,Apr-21-21,01:39PM,Dow Jones Rallies 230 Points As Stock Market R...
4,AAPL,Apr-21-21,01:36PM,4 Dow Jones Stocks To Buy And Watch In April 2...
...,...,...,...,...
95,AAPL,Apr-20-21,04:24AM,"UPDATE 1-Taiwan March export orders up again, ..."
96,AAPL,Apr-20-21,02:20AM,Apple's 'Spring Loaded' Event - Here's What to...
97,AAPL,Apr-20-21,01:52AM,Dow Futures Fall as COVID Surge Clouds US Earn...
98,AAPL,Apr-19-21,06:32PM,Apple to Reinstate Parler; Google Offers Poten...
